In [40]:
import numpy as np
import pandas as pd
import re
import spacy
import string

from datetime import datetime
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler


In [41]:
# Constants
URI = ""
DB_NAME = ""
COLLECTION_NAME = ""

In [42]:
# Get the desired collection.
client = MongoClient(URI, server_api=ServerApi('1'))
db = client[DB_NAME]
writer_info = db[COLLECTION_NAME]

# Prepare the raw dataset.
dataset = {"followers_count": [],
            "publication_following_count": [],
            "has_twitter_username": [],  # convert from string to bool
            "is_writer_program_enrolled": [],
            "allow_notes": [],
            "medium_member_at": [],  # need to calculate the time later
            "is_book_author": [],
            "title": [],  # need to do NLP later
            "subtitle": [],  # need to do NLP later
            #"tags": [],
            #"topics": [],
            "word_count": [],
            "reading_time": [],
            "is_series": [],
            "is_shortform": [],
            "top_highlight": [],  # need to do NLP later
            "content": [],  # need to do NLP later
            "claps": [],
            "voters": [],
            "responses_count": []}

for writer in writer_info.find():
    for article in writer["top_articles"]:
        dataset["followers_count"].append(writer.get("followers_count", 0))
        dataset["publication_following_count"].append(writer.get("publication_following_count", 0))
        dataset["has_twitter_username"].append(writer.get("twitter_username", "").strip() != "")
        dataset["is_writer_program_enrolled"].append(writer.get("is_writer_program_enrolled", False))
        dataset["allow_notes"].append(writer.get("allow_notes", False))
        dataset["medium_member_at"].append(writer.get("medium_member_at", "").strip())
        dataset["is_book_author"].append(writer.get("is_book_author", False))
        dataset["title"].append(article.get("title", "").strip())
        dataset["subtitle"].append(article.get("subtitle", "").strip())
        dataset["word_count"].append(article.get("word_count", 0))
        dataset["reading_time"].append(article.get("reading_time", 0))
        dataset["is_series"].append(article.get("is_series", False))
        dataset["is_shortform"].append(article.get("is_shortform", False))
        dataset["top_highlight"].append(article.get("top_highlight", "").strip())
        dataset["claps"].append(article.get("claps", 0))
        dataset["voters"].append(article.get("voters", 0))
        dataset["responses_count"].append(article.get("responses_count", 0))

        if "content" in article:
            dataset["content"].append(article["content"].get("content", "").strip())
        else:
            dataset["content"].append("")

dataset = pd.DataFrame(dataset)
dataset.shape

(152, 18)

In [43]:
# Convert medium_member_at to medium_usage_time in days.

medium_usage_time = []
current_date = datetime.strptime("2024-02-24 16:45:00", "%Y-%m-%d %H:%M:%S")
for medium_member_at in dataset["medium_member_at"]:
    if medium_member_at == "":
        medium_usage_time.append(0)
        continue

    date = datetime.strptime(medium_member_at, "%Y-%m-%d %H:%M:%S")
    medium_usage_time.append((current_date - date).days)
dataset["medium_usage_time"] = medium_usage_time
dataset.shape

(152, 19)

In [44]:
# Do tfidf to all text fields.
nlp = spacy.load("en_core_web_sm")


def sanitize(texts):
    result = []
    for t in texts:
        text = t.lower()
        text = re.sub('[' + string.punctuation + '0-9\\r\\t\\n]', ' ', text)
        text = re.sub('\\s+', ' ', text)
        text = nlp(text)
        tokens = [words.lemma_ for words in text if not words.is_stop and len(words) >= 2]
        result.append(" ".join(tokens))
    return result

"""
def vectorize(col_name, max_num):
    vectorizer = TfidfVectorizer(stop_words='english', token_pattern=r'\b[a-zA-Z]+\b')
    contents = sanitize(dataset[col_name].values)
    tfidf_matrix = vectorizer.fit_transform(contents)
    feature_names = vectorizer.get_feature_names_out()
    tfidf_sum = np.sum(tfidf_matrix, axis=0)
    tfidf_sum_array = np.squeeze(np.asarray(tfidf_sum))
    tfidf_mapping = dict(zip(feature_names, tfidf_sum_array))
    sorted_tfidf = sorted(tfidf_mapping.items(), key=lambda x: x[1], reverse=True)
    top_words = sorted_tfidf[:max_num]
    indices = [np.where(feature_names == word)[0][0] if word in feature_names else None for word, _ in top_words]
    return tfidf_matrix.toarray()[:, indices]
"""


def vectorize(col_name, max_num):
    vectorizer = CountVectorizer(stop_words='english', token_pattern=r'\b[a-zA-Z]+\b', max_features=max_num)
    contents = sanitize(dataset[col_name].values)  # Assuming sanitize is a function defined elsewhere to clean the data
    bow_matrix = vectorizer.fit_transform(contents)
    feature_names = vectorizer.get_feature_names_out()
    bow_sum = np.sum(bow_matrix, axis=0)
    bow_sum_array = np.squeeze(np.asarray(bow_sum))
    bow_mapping = dict(zip(feature_names, bow_sum_array))
    sorted_bow = sorted(bow_mapping.items(), key=lambda x: x[1], reverse=True)
    top_words = [word for word, _ in sorted_bow[:max_num]]
    indices = [np.where(feature_names == word)[0][0] for word in top_words if word in feature_names]

    return bow_matrix.toarray()[:, indices]


In [45]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

def update_dataset_with_vector(dataset, col_name, max_num):
    dataset_copy = dataset.copy(deep=True)
    vector = vectorize(col_name, max_num)
    n_cols = vector.shape[1]
    for i in range(n_cols):
        dataset_copy[f"{col_name}_{i}"] = vector[:, i]
    return dataset_copy


dataset_final = update_dataset_with_vector(dataset, "title", 50)
dataset_final = update_dataset_with_vector(dataset_final, "subtitle", 50)
dataset_final = update_dataset_with_vector(dataset_final, "top_highlight", 50)
dataset_final = update_dataset_with_vector(dataset_final, "content", 50)
print(dataset_final.shape)

(152, 219)


In [46]:
from enum import Enum


class DataType(Enum):
    TEXT_ONLY = 0
    TEXT_EXCLUSIVE = 1
    ALL_DATA = 2


def get_X(X, data_type=DataType.ALL_DATA):
    non_text_cols = ["followers_count",
                    "publication_following_count",
                    "has_twitter_username",
                    "is_writer_program_enrolled",
                    "allow_notes",
                    "medium_usage_time",
                    "is_book_author",
                    "word_count",
                    "reading_time",
                    "is_series",
                    "is_shortform"]
    X_new = X.copy(deep=True)
    if data_type == DataType.TEXT_ONLY:
        X_new = X_new.drop(columns=non_text_cols)
    elif data_type == DataType.TEXT_EXCLUSIVE:
        X_new = X_new[non_text_cols]
    return X_new


# Try linear regression
def fit_linear_regression(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    model = LinearRegression()
    model.fit(X_train, y_train)
    #y_pred = model.predict(X_test)
    score = model.score(X_test, y_test)
    print(f"Linear Regression R2: {score}")


def fit_random_forest(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    model = RandomForestRegressor(n_estimators=50, max_depth=10,ccp_alpha = 0.1, random_state=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = r2_score(y_test, y_pred)
    print(f"Random Forest R2: {score}")


col_to_exclude = ["medium_member_at", "title", "subtitle", "top_highlight", "content", "claps", "voters", "responses_count"]
X = dataset_final.drop(columns=col_to_exclude)
y_claps = dataset_final["claps"]
y_voters = dataset_final["voters"]
y_responses_count = dataset_final["responses_count"]
fit_linear_regression(X, y_claps)
fit_random_forest(X, y_claps)
fit_linear_regression(X, y_voters)
fit_random_forest(X, y_voters)
fit_linear_regression(X, y_responses_count)
fit_random_forest(X, y_responses_count)

Linear Regression R2: -5.1539585876700915
Random Forest R2: -0.49848399093787465
Linear Regression R2: -86.12345607788389
Random Forest R2: -0.9960375779385318
Linear Regression R2: -30.4181894953775
Random Forest R2: -0.639998402518845


In [47]:
fit_linear_regression(get_X(X, DataType.TEXT_EXCLUSIVE), y_claps)
fit_linear_regression(get_X(X, DataType.TEXT_ONLY), y_claps)
fit_linear_regression(get_X(X, DataType.ALL_DATA), y_claps)
fit_random_forest(get_X(X, DataType.TEXT_EXCLUSIVE), y_claps)
fit_random_forest(get_X(X, DataType.TEXT_ONLY), y_claps)
fit_random_forest(get_X(X, DataType.ALL_DATA), y_claps)

fit_linear_regression(get_X(X, DataType.TEXT_EXCLUSIVE),y_voters)
fit_linear_regression(get_X(X, DataType.TEXT_ONLY), y_voters)
fit_linear_regression(get_X(X, DataType.ALL_DATA), y_voters)
fit_random_forest(get_X(X, DataType.TEXT_EXCLUSIVE), y_voters)
fit_random_forest(get_X(X, DataType.TEXT_ONLY), y_voters)
fit_random_forest(get_X(X, DataType.ALL_DATA), y_voters)

fit_linear_regression(get_X(X, DataType.TEXT_EXCLUSIVE),y_responses_count)
fit_linear_regression(get_X(X, DataType.TEXT_ONLY), y_responses_count)
fit_linear_regression(get_X(X, DataType.ALL_DATA), y_responses_count)
fit_random_forest(get_X(X, DataType.TEXT_EXCLUSIVE), y_responses_count)
fit_random_forest(get_X(X, DataType.TEXT_ONLY), y_responses_count)
fit_random_forest(get_X(X, DataType.ALL_DATA), y_responses_count)

Linear Regression R2: -0.00765671804985546
Linear Regression R2: -8.639614592524273
Linear Regression R2: -157.34656788575552
Random Forest R2: -0.24787309809866764
Random Forest R2: -0.5212665105266716
Random Forest R2: -0.49848399093787465
Linear Regression R2: 0.09314989992237821
Linear Regression R2: -5.859667442244723
Linear Regression R2: -13.221653008252407
Random Forest R2: -0.33432336053336065
Random Forest R2: -0.9292821806806537
Random Forest R2: -0.9960375779385318
Linear Regression R2: -0.18592815951993558
Linear Regression R2: -2.7694406378671044
Linear Regression R2: -29.794874733304002
Random Forest R2: -3.508273576845453
Random Forest R2: -1.5350751230060418
Random Forest R2: -0.639998402518845


In [48]:
y = y_responses_count + y_claps + y_voters


In [49]:
fit_linear_regression(get_X(X, DataType.TEXT_EXCLUSIVE),y)
fit_linear_regression(get_X(X, DataType.TEXT_ONLY), y)
fit_linear_regression(get_X(X, DataType.ALL_DATA), y)
fit_random_forest(get_X(X, DataType.TEXT_EXCLUSIVE), y)
fit_random_forest(get_X(X, DataType.TEXT_ONLY), y)
fit_random_forest(get_X(X, DataType.ALL_DATA), y)

Linear Regression R2: -0.48592464832560145
Linear Regression R2: -40.61016108560024
Linear Regression R2: -4.987688407814307
Random Forest R2: -0.28255120391952593
Random Forest R2: -0.6100022776167733
Random Forest R2: -0.59277713999234
